In [ ]:
# %% [markdown]
# # FASE 2: Preprocesamiento y Limpieza de Texto
# ## Juanes - Procesamiento NLP

# %%
# Instalación de dependencias
import sys
import os
sys.path.append('../scripts')

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from preprocessing import TextPreprocessor

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Cargar datos crudos

# Mostrar información básica del dataset

# Distribución de puntuaciones

# Ejemplo de texto_comentario_comentarios antes del procesamiento

# Inicializar preprocesador

# Procesar una reseña de ejemplo para ver el pipeline

# Procesar todo el dataset

# Calcular métricas adicionales

# Guardar dataset procesado

# Análisis de las métricas calculadas

# Estadísticas de longitud

# Visualización de métricas

# 1. Distribución de longitud de texto_comentario_comentarios originales

# 2. Distribución de longitud de texto_comentario_comentarios limpios

# 3. Distribución de número de palabras

# 4. Densidad léxica por puntuación

# 5. Relación entre longitud y puntuación

# 6. Distribución de densidad léxica

# Análisis de stopwords eliminadas

# Contar frecuencia de palabras antes y después de eliminar stopwords

# Combinar todos los texto_comentario_comentarios

# Contar frecuencias

# Mostrar las palabras más comunes antes de eliminar stopwords

# Preparar datos para la siguiente fase

# Crear dataset optimizado para análisis

# Guardar dataset para análisis

# Resumen del preprocesamiento


: 

In [ ]:
# %% [markdown]
# # FASE 2: Preprocesamiento y Limpieza de Texto
# ## Juanes - Procesamiento NLP


In [ ]:
# %%
import sys
import os
sys.path.append('../scripts')

# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from preprocessing import TextPreprocessor

# Configuración visual
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


In [ ]:
# Cargar datos crudos
ruta_datos = '../data/raw/dataset_raw.csv'
df_raw = pd.read_csv(ruta_datos)

print("INFORMACIÓN DEL DATASET CRUDO")
print("-" * 40)
print(df_raw.info())

print("\nDISTRIBUCIÓN DE PUNTUACIONES:")
print(df_raw['puntuacion'].value_counts().sort_index())



In [ ]:
# Ejemplo de texto antes de procesamiento
ejemplo_texto = df_raw['texto_comentario_comentario'].dropna().iloc[0]

print("TEXTO ORIGINAL:")
print(ejemplo_texto)
